#### Code for introducing Voltage Dependent Theta Neuron Model into Low Rank Connectivity SNNs  


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [7]:
class LowRankSNN:
    # CONSTANTS
    G_L_E = 0.08
    G_L_I = 0.1
    G_P= [0.004069, 0.02672, 0.003276, 0.02138] #g_peak:[E←E, E←I, I←E, I←I]
    V_T = -55
    V_R = -62
    REV_E = 0
    REV_I = -70
    BIAS = V_T - V_R #Bias current
    C0 = 2/(V_T-V_R)
    C1 = (2*REV_E-V_R-V_T)/(V_T-V_R)
    C2 = (2*REV_I-V_R-V_T)/(V_T-V_R)

    def __init__(self,N_E=1000,N_I=200,dt=0.01,T=30,RS=10,IS=100,taud_E=2,taud_I=5) -> None:
        self.N_E = N_E
        self.N_I = N_I
        self.dt = dt
        self.T = T
        self.RS = RS
        self.IS = IS
        self.taud_E = taud_E
        self.taud_I = taud_I
        self.conn = np.zeros((N_E+N_I,N_E+N_I))
        self.added_lowrank = False
        self.added_random = False
        

    def show(self):
        print('Network Settings')
        print('==========================================')
        print('Number of Neurons: ', self.N_E+self.N_I)
        print('Number of Excitatory Units: ', self.N_E)
        print('Number of Inhibitory Units: ', self.N_I)
        # full_w = torch.mm(self.W, self.mask) #包含兴奋和抑制性信息的连接矩阵
        # full_w = self.W * self.mask
        zero_w = (self.conn == 0).sum().item()
        # pos_w = (self.conn > 0).sum().item()
        # neg_w = (full_w < 0 ).sum().item()
        print('Zero Weights occupy: %2.2f %%'%(zero_w/(self.N**2)*100))
        # print('Positive Weights occupy: %2.2f %%'%(pos_w/(self.N**2)*100))
        # print('Negative Weights occupy: %2.2f %%'%(neg_w/(self.N**2)*100))

        #如果有随机连接加low-rank连接就把连接矩阵也画出来

    def add_random(self,conn_rand):
        self.conn_random = conn_rand
        self.conn += conn_rand
        self.added_random = True
        print('Random connection has been added.')
        return

    def remove_random(self):
        if not self.added_random:
            print('There is no random connection added.')
            return
        self.conn -= self.conn_random
        del self.conn_random
        self.added_random = False
        print('Random connection has been removed.')
        return

    def add_lowrank(self,conn_LR, W_out):
        self.conn_lowrank = conn_LR
        self.conn += conn_LR
        self.W_out = W_out
        self.added_lowrank = True
        print('Low Rank connection and readout vector have been added.')
        return

    def remove_lowrank(self):
        if not self.added_lowrank:
            print('There is no Low Rank Connectivity added.')
            return
        self.conn -= self.conn_lowrank
        del self.conn_lowrank
        del self.W_out
        self.added_lowrank = False
        print('Low Rank connection and readout vector have been removed.')

    def V2theta(self,V):
        pass

    def theta2V(self,theta):
        pass

    def simulate(self,Input):
        pass



In [27]:
# Functions
a=np.random.normal(0,1/1000,(1000,1))
b = np.abs(np.random.normal(0,1/1000,(1,1000)))
# b = np.random.normal(0,1/1000,(1,1000))
c = np.dot(b,a)

In [8]:
# Initialiazation
LRSNN = LowRankSNN(N_E=1000,N_I=200,dt=0.01,T=30,RS=10,IS=100,taud_E=2,taud_I=5)

# Go_NoGo Task
# Prepare the Stimuli and Readout Vector
temp = np.random.rand(1,LRSNN.N_E+LRSNN.N_I) #Size (1,N_E) for Sti_go and nogo #这里我想试试把Low Rank加到整个网络上
# print(temp)
Sti_go = temp.copy()
Sti_nogo = temp.copy()
W_out = temp.copy()
Sti_go[Sti_go>1/3] = 0
Sti_nogo[Sti_nogo<1/3] = 0
Sti_nogo[Sti_nogo>2/3] = 0
W_out[W_out<2/3] = 0
# print(np.nonzero(Sti_go)[0])
# print(len(np.nonzero(Sti_go)[0]))
Sti_go[Sti_go!=0] = np.random.gamma(1,1,len(np.nonzero(Sti_go)[0])) #random.gamma(shape, scale=1.0, size=None),这个地方的Gamma分布及其参数选取需要进一步讨论
Sti_nogo[Sti_nogo!=0] = np.random.gamma(1,1,len(np.nonzero(Sti_nogo)[0]))
W_out[W_out!=0] = np.random.gamma(1,1,len(np.nonzero(W_out)[0]))
W_out = np.transpose(W_out) #Size (N_E,1)
# Low Rank Connectivity (Rank = 1)
conn_LR = W_out*Sti_go/(LRSNN.N_E+LRSNN.N_I) # 为什么除以神经元总数?
# Random Connectivity
conn_rand = np.random.gamma(1,1,(LRSNN.N_E+LRSNN.N_I,LRSNN.N_E+LRSNN.N_I)) #这里的Gamma分布取值也需要讨论

# Assemble the Network
LRSNN.add_lowrank(conn_LR, W_out)
LRSNN.add_random(conn_rand)

# Show the Network information before simulaiton




Low Rank connection and readout vector have been added.
Random connection has been added.


In [ ]:
# Simulation



In [ ]:
# Draw the results

In [1]:
print({'a':1,'b':2})

{'a': 1, 'b': 2}
